In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [2]:
import sys
sys.path.insert(1, '../oracle-polimi-contest-2019')
from evaluation_script import read_file

In [3]:
import similaripy

In [134]:
train = read_file("../oracle-polimi-contest-2019/entity-resolution_advanced-topics-training_data.csv")
test = read_file("../oracle-polimi-contest-2019/test_data.csv")

In [135]:
train = train.drop(['modification', 'type'], axis=1)
test = test.drop(['modification', 'type'], axis=1)
train['name'] = train['name'].str.lower()
test['name'] = test['name'].str.lower()
train

,record_id,name,address,phone,email,linked_id
0,10111147-T1,ardia enterprisesmltd.,NaN,+4102971003602,imquires@zoho.vo,10111147
1,12172071-T0,andré almeida blanco,la Rua Pelotas; 209; apartamento 72; Bloco A; ...,NaN,NaN,12172071
2,10104491-T1,moonta holdings ltd.,NaN,+65013464145,NaN,10104491
3,10068808,springfield investments group limited,NaN,NaN,consulting@zoho.de,10068808
4,13011715,grigory uvarov,127273; MOSCOW; OTRADNAJA STREET; HOUSE 2; NO....,NaN,NaN,13011715
5,10105687,odin group investments s.a.,NaN,+49668906658,info@outlook.cz,10105687
6,10016805-T2,capri solutions limited,NaN,NaN,help@CAPRI.SOLUTIONS.LxMITED.de,10016805
7,12133421,crossmatch s.a.,"Akara building 24 De Castro Street, Wickham Ca...",NaN,CrossmatchSA@aol.gov,12133421
8,12126772,laurence pountney ltd,Akara Building 24 De Castro St; Wickham Cay I;...,NaN,support@LAURENCE.POUNTNEY.LTD.gov,12126772
9,11002523,"breit, marcus",NaN,+440180275310,NaN,11002523


In [6]:
train.loc[438728]

record_id                     10127893-T1
name         juno star investment limited
address                               NaN
phone                      +0195361657938
email                colpulting@icaoud.cz
linked_id                        10127893
Name: 438728, dtype: object

In [7]:
test.shape

(266955, 5)

In [8]:
target = train.linked_id
#train = train.drop('linked_id', axis=1)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.33, random_state=42)

In [10]:
X_val.shape

(228176, 6)

In [11]:
# Cosine similarity on validation set

In [12]:
import string
import unidecode
def create_name_letters_matrix(df):
    df = df[['record_id','name']]
    df.name = df.name.astype(str) # convert to string
    df.name = df.name.str.lower() # lowercase
    df.name = df.name.str.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
    # remove accented letters
    no_accents = []
    for s in df.name:
        no_accents.append(unidecode.unidecode(s))
    df.name = no_accents
    # create return matrix
    columns = ['record_id','name','a','b','c','d','e','f','g','h','i','j','k','l',
               'm','n','o','p','q','r','s','t','u','v','w','x','y','z']
    name_letters_matrix = pd.DataFrame(columns=columns)
    name_letters_matrix.record_id = df.record_id.copy()
    name_letters_matrix.name = df.name.copy()
    # count occurence of each letter and add the columns to the return df
    for l in tqdm(['a','b','c','d','e','f','g','h','i','j','k','l','m','n',
                   'o','p','q','r','s','t','u','v','w','x','y','z']):
        new_col = []
        for (i,n) in zip(name_letters_matrix.index, name_letters_matrix.name):
            new_col.append(n.count(l))
        name_letters_matrix[l] = new_col
    return name_letters_matrix

In [14]:
group = X_train[['name', 'linked_id']].groupby('linked_id').apply(lambda x: list(x['name']))

In [15]:
from collections import Counter
link_mc_name = {}
for (l, names) in tqdm(zip(group.keys(), group)):
    link_mc_name[l] = Counter(names).most_common(1)[0][0]

link_mc_name

{'10000001': 'tiansheng industry and trading co ltd',
 '10000002': 'ningbo sunrise enterprises united co ltd',
 '10000004': 'skyblue gifts  toys co ltd',
 '10000005': 'fortunemaker investments corporation',
 '10000006': '8808 holding limited',
 '10000007': 'kent development limited',
 '10000009': 'amarandan ltd',
 '10000010': 'new idea limited',
 '10000011': 'hugh power limited',
 '10000014': 'charter mark limited',
 '10000015': 'wide international trading co ltd',
 '10000018': 'chem dt corp',
 '10000019': 'htss et capital limited',
 '10000020': 'jie lun investment limited',
 '10000021': 'fortune palace limited',
 '10000022': 'prestige international corp',
 '10000023': 'sunny treasure international limited',
 '10000027': 'world tools investments corp',
 '10000029': 'necy consultants limited',
 '10000031': 'aegis infocom inc',
 '10000032': 'topline investments limited',
 '10000033': 'clarksburg management ltd',
 '10000035': 'carpenter nelson  co ltd',
 '10000036': 'riviera holdings limi

In [17]:
most_common_name = pd.DataFrame.from_dict(link_mc_name, orient='index', columns=['most_common_name'])

df_train_clean = pd.merge(X_train, most_common_name, how='left', left_on='linked_id', right_index=True)
df_train_clean = df_train_clean.drop_duplicates(subset=['linked_id','most_common_name']).drop(['record_id', 'name'], axis=1)
df_train_clean = df_train_clean.rename(columns={"linked_id":"record_id", "most_common_name":"name"})
m_train = create_name_letters_matrix(df_train_clean)
m_train

,record_id,name,a,b,c,d,e,f,g,h,...,q,r,s,t,u,v,w,x,y,z
173002,10098822,fromley trading inc,1,0,1,1,1,1,1,0,...,0,2,0,1,0,0,0,0,1,0
467634,10074742,radiant investment management corp,4,0,1,1,4,0,1,0,...,0,2,1,4,0,1,0,0,0,0
656895,10190457,rosemead ltd,1,0,0,2,2,0,0,0,...,0,1,1,1,0,0,0,0,0,0
186071,10187369,sperry corp,0,0,1,0,1,0,0,0,...,0,3,1,0,0,0,0,0,1,0
273645,12141090,isidoro toledo esquenazi,1,0,0,2,3,0,0,0,...,1,1,2,1,1,0,0,0,0,1
434416,12165506,gabriel davidov pardo,3,1,0,3,1,0,1,0,...,0,2,0,0,0,2,0,0,0,0
346780,12207761,hhoutianhua,2,0,0,0,0,0,0,3,...,0,0,0,1,2,0,0,0,0,0
403129,12134974,tian quan enterprises limited,2,0,0,1,4,0,0,0,...,1,2,2,3,1,0,0,0,0,0
355795,10211979,fundacion allan rausch,4,0,2,1,0,1,0,1,...,0,1,1,0,2,0,0,0,0,0
550,10213246,zinger international gmbh,2,1,0,0,2,0,2,1,...,0,2,0,2,0,0,0,0,0,1


In [19]:
m_test = create_name_letters_matrix(X_val)

In [20]:
m_train = m_train.reset_index(drop = True)

In [21]:
from scipy import *
from scipy.sparse import *
import numpy as np
m_train_csr = csr_matrix(m_train.drop(['record_id','name'], axis=1))
m_test_csr = csr_matrix(m_test.drop(['record_id','name'], axis=1))

In [22]:
import similaripy as sim
output = sim.cosine(m_test_csr, m_train_csr.T, k=10)
save_npz('val_set_cosine.npz', output.tocsr())
#output = load_npz("../val_set_cosine.npz")

Done: 100%|██████████| 228176/228176 [16:57<00:00, 224.26it/s]            


In [24]:
output = output.tocsr()
r_nnz = output.nonzero()[0]
c_nnz = output.nonzero()[1]

l = []
for i in tqdm(range(len(r_nnz))):
    l.append([output[r_nnz[i], c_nnz[i]],r_nnz[i],c_nnz[i]])

In [25]:
l

[[1.0, 0, 43891],
 [0.9844251, 0, 79338],
 [0.9772655, 0, 103446],
 [0.9764168, 0, 103898],
 [0.9751974, 0, 141165],
 [0.9764168, 0, 152905],
 [0.9764168, 0, 155486],
 [0.9844251, 0, 179531],
 [0.9764168, 0, 183062],
 [0.9752227, 0, 201500],
 [0.94299036, 1, 838],
 [0.94174194, 1, 61304],
 [0.93094933, 1, 82531],
 [0.9302605, 1, 107049],
 [0.9460998, 1, 114474],
 [1.0, 1, 131280],
 [0.94299036, 1, 141059],
 [0.9544271, 1, 156612],
 [0.9460998, 1, 162988],
 [0.9302605, 1, 220546],
 [0.9582877, 2, 53161],
 [0.95827484, 2, 96975],
 [0.95197165, 2, 111894],
 [0.9647343, 2, 136995],
 [0.9535996, 2, 139464],
 [0.96025336, 2, 159107],
 [0.9573414, 2, 165176],
 [0.95124173, 2, 183677],
 [0.9826073, 2, 187284],
 [0.9582877, 2, 213158],
 [0.9686649, 3, 39410],
 [0.99999994, 3, 66160],
 [0.9648363, 3, 135589],
 [0.9460998, 3, 136476],
 [0.9460998, 3, 149153],
 [0.9534626, 3, 154158],
 [0.9594032, 3, 156997],
 [0.9468419, 3, 157367],
 [0.9557916, 3, 166551],
 [0.9574271, 3, 208431],
 [1.0000001, 4

In [26]:
l.sort(key= lambda x: (x[1], -x[0]))
rec_id = [x[1] for x in l]
rec_id[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [29]:
X_val = X_val.reset_index(drop=True) #aggiunto

In [ ]:
X_val

In [30]:
rec_id = [X_val.at[i,'record_id'] for i in tqdm(rec_id)]
rec_id

['10127893-T1',
 '10127893-T1',
 '10127893-T1',
 '10127893-T1',
 '10127893-T1',
 '10127893-T1',
 '10127893-T1',
 '10127893-T1',
 '10127893-T1',
 '10127893-T1',
 '12187326-M1',
 '12187326-M1',
 '12187326-M1',
 '12187326-M1',
 '12187326-M1',
 '12187326-M1',
 '12187326-M1',
 '12187326-M1',
 '12187326-M1',
 '12187326-M1',
 '10196371-T0',
 '10196371-T0',
 '10196371-T0',
 '10196371-T0',
 '10196371-T0',
 '10196371-T0',
 '10196371-T0',
 '10196371-T0',
 '10196371-T0',
 '10196371-T0',
 '10102093-NV2',
 '10102093-NV2',
 '10102093-NV2',
 '10102093-NV2',
 '10102093-NV2',
 '10102093-NV2',
 '10102093-NV2',
 '10102093-NV2',
 '10102093-NV2',
 '10102093-NV2',
 '10025744',
 '10025744',
 '10025744',
 '10025744',
 '10025744',
 '10025744',
 '10025744',
 '10025744',
 '10025744',
 '10025744',
 '10109220',
 '10109220',
 '10109220',
 '10109220',
 '10109220',
 '10109220',
 '10109220',
 '10109220',
 '10109220',
 '10109220',
 '10114863',
 '10114863',
 '10114863',
 '10114863',
 '10114863',
 '10114863',
 '10114863',

In [31]:
lin_id = [x[2] for x in l]
lin_id[:20]

[43891,
 79338,
 179531,
 103446,
 103898,
 152905,
 155486,
 183062,
 201500,
 141165,
 131280,
 156612,
 114474,
 162988,
 838,
 141059,
 61304,
 82531,
 107049,
 220546]

In [32]:
lin_id = [m_train.at[i,'record_id'] for i in tqdm(lin_id)]
lin_id

['10127893',
 '10019372',
 '10211614',
 '10058787',
 '10181531',
 '10117117',
 '10175281',
 '10138368',
 '10108944',
 '10185271',
 '12187326',
 '12163761',
 '12160567',
 '12106010',
 '12224380',
 '12126322',
 '13002713',
 '12170067',
 '12117467',
 '13012781',
 '10196371',
 '12012875',
 '10075715',
 '10055937',
 '10052995',
 '10095054',
 '10185950',
 '10173245',
 '10183448',
 '10131695',
 '10102093',
 '10140502',
 '10193702',
 '10176410',
 '10059876',
 '10103826',
 '10198250',
 '10150071',
 '10059589',
 '10143310',
 '10025744',
 '10004165',
 '12207650',
 '10003036',
 '10103917',
 '10023073',
 '10150768',
 '10207337',
 '10022734',
 '10018830',
 '10068751',
 '12189933',
 '10182089',
 '10104528',
 '10094332',
 '10001031',
 '10170526',
 '10100545',
 '10204134',
 '10157154',
 '10114863',
 '10179434',
 '10194113',
 '10057261',
 '10107987',
 '10060569',
 '10153086',
 '12202807',
 '10053765',
 '10038772',
 '10040402',
 '10083924',
 '10166580',
 '12010196',
 '10067542',
 '12043229',
 '10152915',

In [33]:
scores = [x[0] for x in l]
scores[:20]

[1.0,
 0.9844251,
 0.9844251,
 0.9772655,
 0.9764168,
 0.9764168,
 0.9764168,
 0.9764168,
 0.9752227,
 0.9751974,
 1.0,
 0.9544271,
 0.9460998,
 0.9460998,
 0.94299036,
 0.94299036,
 0.94174194,
 0.93094933,
 0.9302605,
 0.9302605]

In [34]:
df = pd.DataFrame()
df['queried_record_id'] = rec_id
df['predicted_record_id'] = lin_id
df['cosine_score'] = scores
df

,queried_record_id,predicted_record_id,score
0,10127893-T1,10127893,1.000000
1,10127893-T1,10019372,0.984425
2,10127893-T1,10211614,0.984425
3,10127893-T1,10058787,0.977265
4,10127893-T1,10181531,0.976417
5,10127893-T1,10117117,0.976417
6,10127893-T1,10175281,0.976417
7,10127893-T1,10138368,0.976417
8,10127893-T1,10108944,0.975223
9,10127893-T1,10185271,0.975197


In [ ]:
#group_sub = subbb.groupby('queried_record_id').apply(lambda x: list(x['predicted_record_id']))
#group_sub
#subbbbbbb = pd.DataFrame(group_sub)
#subbbbbbb = subbbbbbb.reset_index()
#subbbbbbb = subbbbbbb.rename(columns={0: 'predicted_record_id'})
#new_col = []
#for t in tqdm(subbbbbbb.predicted_record_id):
#    new_col.append(' '.join([str(x) for x in t]))
    
#subbbbbbb.predicted_record_id = new_col
#subbbbbbb.to_csv("cossim10top.csv", index=False)

In [35]:
X_val

,record_id,name,address,phone,email,linked_id
0,10127893-T1,juno star investment limited,NaN,+0195361657938,colpulting@icaoud.cz,10127893
1,12187326-M1,"hung, yu-chi",No.35 Aly.28 Lane43 Sec.3 Ta Feng Rd.; Tanzi D...,NaN,NaN,12187326
2,10196371-T0,broadoak investments lidited,NaN,+8154367225100,NaN,10196371
3,10102093-NV2,sandcastle trading ltd,NaN,+49965206548,info@hotmail.com,10102093
4,10025744,"ontime electronic technology co., ltd",NaN,NaN,NaN,10025744
5,10109220,trehearne limited,NaN,NaN,NaN,10109220
6,10114863,eastwood holdings development ltd.,NaN,+18507279781,NaN,10114863
7,10040402-T3,mountain properties s.a,NaN,+420924287337,NaN,10040402
8,12153582,ivone riesna zakaria,D.H.I BARAT 9/9 BLOK-A/301-302 RT/RW 009/005 S...,+3923572620,IvoneRiesnaZakaria@hotmail.it,12153582
9,12096823-T3,itay barak,12 Bney Dror St.; Bney Dror; Israel,+336259154021,NaN,12096823


In [ ]:
# Adding editdistance between queried_id.name and predicted_record_id.name

In [69]:
df_new = df.merge(m_train[['record_id', 'name']], left_on='predicted_record_id', right_on='record_id').drop('record_id', axis=1)
df_new = df_new.rename(columns={'name': 'predicted_record_name'})
df_new = df_new.merge(m_test[['record_id', 'name']], left_on='queried_record_id', right_on='record_id' ).rename(columns={'name':'queried_name'})
df_new = df_new.drop('record_id', axis=1)

df_new = df_new.merge(X_val[['record_id', 'linked_id']], left_on='queried_record_id', right_on='record_id')
df_new = df_new.drop('record_id', axis=1)

df_new['predicted_record_id'] = df_new['predicted_record_id'].astype(int)
df_new['linked_id'] = df_new['linked_id'].astype(int)
df_new['score'] = df_new['score'].astype(float)

df_new

,queried_record_id,predicted_record_id,score,name
0,10127893-T1,10127893,1.000000,juno star investment limited
1,10138084,10127893,0.955608,juno star investment limited
2,10205627-NV1,10127893,0.971726,juno star investment limited
3,10138084-NV0,10127893,0.955608,juno star investment limited
4,10077355-M0,10127893,0.902244,juno star investment limited
5,10166681,10127893,0.974913,juno star investment limited
6,10067795,10127893,0.975809,juno star investment limited
7,10019372-NV0,10127893,0.984425,juno star investment limited
8,10019372,10127893,0.984425,juno star investment limited
9,10127893-T1,10019372,0.984425,oruna investments limited


In [75]:
def extract_target(predicted, linked):
    res = np.empty(len(predicted))
    res = np.where(predicted == linked, 1, 0)
    return res

In [76]:
df_new['target'] = extract_target(df_new.predicted_record_id.values, df_new.linked_id.values)
df_new

,queried_record_id,predicted_record_id,score,predicted_record_name,queried_name,linked_id,target
0,10127893-T1,10127893,1.000000,juno star investment limited,juno star investment limited,10127893,1
1,10127893-T1,10019372,0.984425,oruna investments limited,juno star investment limited,10127893,0
2,10127893-T1,10211614,0.984425,noura investments limited,juno star investment limited,10127893,0
3,10127893-T1,10058787,0.977265,mountain rose investments limited,juno star investment limited,10127893,0
4,10127893-T1,10181531,0.976417,ariton investments limited,juno star investment limited,10127893,0
5,10127893-T1,10117117,0.976417,truina investments limited,juno star investment limited,10127893,0
6,10127893-T1,10175281,0.976417,nature investments limited,juno star investment limited,10127893,0
7,10127893-T1,10138368,0.976417,tournado investments limited,juno star investment limited,10127893,0
8,10127893-T1,10108944,0.975223,super nation investment limited,juno star investment limited,10127893,0
9,10127893-T1,10185271,0.975197,lauriston investments limited,juno star investment limited,10127893,0


In [84]:
import editdistance
import fuzzy

In [79]:
def extract_editdistance(queried_name, predicted_name):
    res = np.empty(len(queried_name))
    for i in tqdm(range(len(queried_name))):
        res[i] = editdistance.eval(queried_name[i], predicted_name[i])
    return res

In [80]:
df_new['editdistance'] = extract_editdistance(df_new.predicted_record_name.values, df_new.queried_name.values)

In [82]:
df_new['editdistance'] = df_new['editdistance'].astype(int)

In [85]:
def extract_nysiis_distance(queried_name, predicted_name):
    res = np.empty(len(queried_name))
    for i in tqdm(range(len(queried_name))):
        q_nysiis = fuzzy.nysiis(queried_name[i])
        p_nysiis = fuzzy.nysiis(predicted_name[i])
        res[i] = editdistance.eval(q_nysiis, p_nysiis)
    return res

In [88]:
df_new['nysiis_distance'] = extract_editdistance(df_new.predicted_record_name.values, df_new.queried_name.values).astype(int)
# Non ha senso, ha la stessa distanza di edit distance

In [95]:
df_xgb = df_new[['queried_record_id', 'predicted_record_id', 'score', 'editdistance', 'nysiis_distance', 'target']]
df_xgb

,queried_record_id,predicted_record_id,score,editdistance,nysiis_distance,target
0,10127893-T1,10127893,1.000000,0,0,1
1,10127893-T1,10019372,0.984425,8,8,0
2,10127893-T1,10211614,0.984425,7,7,0
3,10127893-T1,10058787,0.977265,11,11,0
4,10127893-T1,10181531,0.976417,9,9,0
5,10127893-T1,10117117,0.976417,9,9,0
6,10127893-T1,10175281,0.976417,8,8,0
7,10127893-T1,10138368,0.976417,9,9,0
8,10127893-T1,10108944,0.975223,9,9,0
9,10127893-T1,10185271,0.975197,8,8,0


In [102]:
diff = np.where(df_xgb.editdistance.values != df_xgb.nysiis_distance.values, 1, 0)
diff

array([1, 1, 1, ..., 1, 1, 1])

In [103]:
diff.sum()

2281740

In [99]:
group = df_xgb.groupby('queried_record_id').size().values

In [94]:
import xgboost as xgb

In [105]:
ranker = xgb.XGBRanker()
ranker.fit(df_xgb.drop(['queried_record_id', 'target', 'nysiis_distance'], axis=1), df_xgb['target'], group=group)

/usr/local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
          max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
          n_jobs=-1, nthread=None, objective='rank:pairwise', random_state=0,
          reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
          subsample=1)

In [106]:
######### TEST BUILDING ######

In [128]:
#test_sub = pd.read_csv("/Users/alessiorussointroito/Downloads/sub_new.csv")
#test

,queried_record_id,predicted_record_id
0,10000003-TST-MR,10097822 10022487 10090422 10002385 10113802 1...
1,10000008-TST-M,10043172 10211873 10080769 10186437 10029584 1...
2,10000010-TST-CP,10000010 10198226 10021610 10108279 10186594 1...
3,10000013-TST-MR,10153043 10147826 10189807 10027275 10071664 1...
4,10000016-TST-MR,10002588 10147072 10034520 10006940 10012410 1...
5,10000017-TST-MR,10196127 10087602 10126538 10152573 12131717 1...
6,10000018-T4-TST-CP,10000018 10100274 10000069 12180349 10160621 1...
7,10000018-TST-M,10000018 10100274 10000069 12180349 10160621 1...
8,10000020-NV0-TST-CP,10000020 10070569 10094948 10140964 10128438 1...
9,10000020-NV1-TST-M,10000020 10070569 10094948 10140964 10128438 1...


In [129]:
def expand_submission(df):
    df_list = []
    for (i, records) in tqdm(zip(df.queried_record_id, df.predicted_record_id)):
        for x in records:
            df_list.append((i, x))
    df = pd.DataFrame(df_list, columns=['queried_record_id', 'predicted_record_id'])
    return df

In [130]:
#test['predicted_record_id'] = test['predicted_record_id'].str.split(' ')
#test = expand_submission(test)

In [133]:
# Extract Cosine score for test

In [136]:
test

,record_id,name,address,phone,email
0,10051937-TST-MR,cheyham corporation limited,NaN,+19865374390,sales@protonmail.com
1,10070762-TST-MR,meabiz limited,NaN,+337403339169,NaN
2,10000304-TST-MR,chample investments ltd.,NaN,+4142094239081,NaN
3,10151607-TST-MR,concept i media and technology company limited,NaN,+14803535251,NaN
4,10124701-TST-MR,arra resources limited,NaN,+420762950520,NaN
5,10140018-TST-MR,ronco investment estates corp.,NaN,+332637199890,NaN
6,10012709-TST-MR,miren enterprises s.a.,NaN,NaN,MIRENENTERPRISESSA@outlook.it
7,10127252-TST-MR,white concept group limited,NaN,+18013438855,NaN
8,12167036-TST-MR,zhang zi tong,No. 102; Unit 11; 5th Floor; Tu Er hu tong; Do...,+420977095071,ZHANGZITONG@icloud.de
9,10088906-TST-MR,freshcam corp.,NaN,NaN,NaN


In [138]:
df_train = train.copy()
df_train

,record_id,name,address,phone,email,linked_id
0,10111147-T1,ardia enterprisesmltd.,NaN,+4102971003602,imquires@zoho.vo,10111147
1,12172071-T0,andré almeida blanco,la Rua Pelotas; 209; apartamento 72; Bloco A; ...,NaN,NaN,12172071
2,10104491-T1,moonta holdings ltd.,NaN,+65013464145,NaN,10104491
3,10068808,springfield investments group limited,NaN,NaN,consulting@zoho.de,10068808
4,13011715,grigory uvarov,127273; MOSCOW; OTRADNAJA STREET; HOUSE 2; NO....,NaN,NaN,13011715
5,10105687,odin group investments s.a.,NaN,+49668906658,info@outlook.cz,10105687
6,10016805-T2,capri solutions limited,NaN,NaN,help@CAPRI.SOLUTIONS.LxMITED.de,10016805
7,12133421,crossmatch s.a.,"Akara building 24 De Castro Street, Wickham Ca...",NaN,CrossmatchSA@aol.gov,12133421
8,12126772,laurence pountney ltd,Akara Building 24 De Castro St; Wickham Cay I;...,NaN,support@LAURENCE.POUNTNEY.LTD.gov,12126772
9,11002523,"breit, marcus",NaN,+440180275310,NaN,11002523


In [139]:
from collections import Counter

group = df_train[['name', 'linked_id']].groupby('linked_id').apply(lambda x: list(x['name']))
link_mc_name = {}
for (l, names) in tqdm(zip(group.keys(), group)):
    link_mc_name[l] = Counter(names).most_common(1)[0][0]

link_mc_name

{'10000001': 'tiansheng industry and trading co., ltd.',
 '10000002': 'ningbo sunrise enterprises united co., ltd.',
 '10000004': 'sky-blue gifts & toys co., ltd.',
 '10000005': 'fortunemaker investments corporation',
 '10000006': '8808 holding limited',
 '10000007': 'kent development limited',
 '10000009': 'amarandan ltd.',
 '10000010': 'new idea limited',
 '10000011': 'hugh power limited',
 '10000012': 'ultimate group limited',
 '10000014': 'charter mark limited',
 '10000015': 'wide international trading co., ltd.',
 '10000018': 'chem d-t corp.',
 '10000019': 'htss et capital limited',
 '10000020': 'jie lun investment limited',
 '10000021': 'fortune palace limited',
 '10000022': 'prestige international corp.',
 '10000023': 'sunny treasure international limited',
 '10000026': 'euro investment capital inc.',
 '10000027': 'world tools investments corp.',
 '10000029': 'necy consultants limited',
 '10000031': 'aegis infocom inc.',
 '10000032': 'topline investments limited',
 '10000033': '

In [140]:
most_common_name = pd.DataFrame.from_dict(link_mc_name, orient='index', columns=['most_common_name'])
df_train_clean = pd.merge(df_train, most_common_name, how='left', left_on='linked_id', right_index=True)
df_train_clean = df_train_clean.drop_duplicates(subset=['linked_id','most_common_name']).drop(['record_id', 'name'], axis=1)
df_train_clean = df_train_clean.rename(columns={"linked_id":"record_id", "most_common_name":"name"})
m_train = create_name_letters_matrix(df_train_clean)
m_train

/Users/alessiorussointroito/Library/Python/3.7/lib/python/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,record_id,name,a,b,c,d,e,f,g,h,...,q,r,s,t,u,v,w,x,y,z
0,10111147,aldia enterprises ltd,2,0,0,2,3,0,0,0,...,0,2,2,2,0,0,0,0,0,0
1,12172071,andre almeida blanco,4,1,1,2,2,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,10104491,moonta holdings ltd,1,0,0,2,0,0,1,1,...,0,0,1,2,0,0,0,0,0,0
3,10068808,springfield investments group limited,0,0,0,2,4,1,2,0,...,0,2,3,3,1,1,0,0,0,0
4,13011715,grigory uvarov,1,0,0,0,0,0,2,0,...,0,3,0,0,1,2,0,0,1,0
5,10105687,odin group investments sa,1,0,0,1,2,0,1,0,...,0,1,3,2,1,1,0,0,0,0
6,10016805,capri solutions limited,1,0,1,1,1,0,0,0,...,0,1,2,2,1,0,0,0,0,0
7,12133421,crossmatch sa,2,0,2,0,0,0,0,1,...,0,1,3,1,0,0,0,0,0,0
8,12126772,laurence pountney ltd,1,0,1,1,3,0,0,0,...,0,1,0,2,2,0,0,0,1,0
9,11002523,breit marcus,1,1,1,0,1,0,0,0,...,0,2,1,1,1,0,0,0,0,0


In [142]:
m_train = m_train.reset_index(drop=True)

In [141]:
df_test = test.copy()

In [143]:
m_test = create_name_letters_matrix(df_test)

In [144]:
m_test

,record_id,name,a,b,c,d,e,f,g,h,...,q,r,s,t,u,v,w,x,y,z
0,10051937-TST-MR,cheyham corporation limited,2,0,2,1,2,0,0,2,...,0,2,0,2,0,0,0,0,1,0
1,10070762-TST-MR,meabiz limited,1,1,0,1,2,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,10000304-TST-MR,chample investments ltd,1,0,1,1,3,0,0,1,...,0,0,2,3,0,1,0,0,0,0
3,10151607-TST-MR,concept i media and technology company limited,3,0,4,3,4,0,1,1,...,0,0,0,3,0,0,0,0,2,0
4,10124701-TST-MR,arra resources limited,2,0,1,1,3,0,0,0,...,0,4,2,1,1,0,0,0,0,0
5,10140018-TST-MR,ronco investment estates corp,1,0,2,0,4,0,0,0,...,0,2,3,4,0,1,0,0,0,0
6,10012709-TST-MR,miren enterprises sa,1,0,0,0,4,0,0,0,...,0,3,3,1,0,0,0,0,0,0
7,10127252-TST-MR,white concept group limited,0,0,2,1,3,0,1,1,...,0,1,0,3,1,0,1,0,0,0
8,12167036-TST-MR,zhang zi tong,1,0,0,0,0,0,2,1,...,0,0,0,1,0,0,0,0,0,2
9,10088906-TST-MR,freshcam corp,1,0,2,0,1,1,0,1,...,0,2,1,0,0,0,0,0,0,0


In [ ]:
m_train_csr = csr_matrix(m_train.drop(['record_id','name'], axis=1))
m_test_csr = csr_matrix(m_test.drop(['record_id','name'], axis=1))
output = sim.cosine(m_test_csr, m_train_csr.T, k=10)
save_npz('similarity_cosine_2.npz', output.tocsr())

In [173]:
output = load_npz("/Users/alessiorussointroito/Downloads/Telegram Desktop/similarity_cosine.npz")

In [174]:
output = output.tocsr()
r_nnz = output.nonzero()[0]
c_nnz = output.nonzero()[1]

l = []
for i in tqdm(range(len(r_nnz))):
    l.append([output[r_nnz[i], c_nnz[i]],r_nnz[i],c_nnz[i]])
    
l.sort(key= lambda x: (x[1], -x[0]))

In [175]:
rec_id_test = [x[1] for x in l]
rec_id_test = [df_test.at[i,'record_id'] for i in tqdm(rec_id_test)]

In [176]:
lin_id_test = [x[2] for x in l]

In [178]:
lin_id_test = [m_train.at[i,'record_id'] for i in tqdm(lin_id_test)]

In [179]:
scores_test = [x[0] for x in l]

In [180]:
df_test_new = pd.DataFrame()
df_test_new['queried_record_id'] = rec_id_test
df_test_new['predicted_record_id'] = lin_id_test
df_test_new['cosine_score'] = scores_test
df_test_new

,queried_record_id,predicted_record_id,cosine_score
0,10051937-TST-MR,10110955,0.968152
1,10051937-TST-MR,10117193,0.956102
2,10051937-TST-MR,12212206,0.956102
3,10051937-TST-MR,12106617,0.952661
4,10051937-TST-MR,10036617,0.952661
5,10051937-TST-MR,10040198,0.952036
6,10051937-TST-MR,10203106,0.950654
7,10051937-TST-MR,10136888,0.947255
8,10051937-TST-MR,10211735,0.946762
9,10051937-TST-MR,10145409,0.945905


In [181]:
df_test_new = df_test_new.merge(m_train[['record_id', 'name']], left_on='predicted_record_id', right_on='record_id')
df_test_new = df_test_new.drop('record_id', axis=1)
df_test_new = df_test_new.rename(columns={'name': 'predicted_name'})
df_test_new

,queried_record_id,predicted_record_id,cosine_score,name
0,10051937-TST-MR,10110955,0.968152,h ch laboratories company limited
1,12108576-TST-MR,10110955,0.970645,h ch laboratories company limited
2,12173621-TST-MR,10110955,0.896848,h ch laboratories company limited
3,10031055-TST-MR,10110955,0.900125,h ch laboratories company limited
4,12110407-TST-M,10110955,0.907207,h ch laboratories company limited
5,12212206-M1-TST-M,10110955,0.991912,h ch laboratories company limited
6,12106493-T2-TST-CP,10110955,0.955657,h ch laboratories company limited
7,10101237-TST-CP,10110955,0.949698,h ch laboratories company limited
8,10051937-TST-MR,10117193,0.956102,c h laboratories company limited
9,10182751-TST-MR,10117193,0.937715,c h laboratories company limited


In [182]:
df_test_new = df_test_new.merge(m_test[['record_id','name']], left_on='queried_record_id', right_on='record_id').drop('record_id', axis=1)
df_test_new = df_test_new.rename(columns={'name': 'queried_name'})
df_test_new

,queried_record_id,predicted_record_id,cosine_score,name_x,name_y
0,10051937-TST-MR,10110955,0.968152,h ch laboratories company limited,cheyham corporation limited
1,10051937-TST-MR,10117193,0.956102,c h laboratories company limited,cheyham corporation limited
2,10051937-TST-MR,12212206,0.956102,ch laboratories company limited,cheyham corporation limited
3,10051937-TST-MR,12106617,0.952661,aprontech co limited,cheyham corporation limited
4,10051937-TST-MR,10036617,0.952661,dehanchar corporation limited,cheyham corporation limited
5,10051937-TST-MR,10040198,0.952036,rich hope company limited,cheyham corporation limited
6,10051937-TST-MR,10203106,0.950654,heworth company limited,cheyham corporation limited
7,10051937-TST-MR,10136888,0.947255,shiruida machinery import export co ltd,cheyham corporation limited
8,10051937-TST-MR,10211735,0.946762,hareton company limited,cheyham corporation limited
9,10051937-TST-MR,10145409,0.945905,pastry horizon china co limited,cheyham corporation limited


In [183]:
#df_test_new = df_test_new.rename(columns={'name_x': 'predicted_name', 'name_y': 'queried_name'})
#df_test_new

,queried_record_id,predicted_record_id,cosine_score,predicted_name,queried_name
0,10051937-TST-MR,10110955,0.968152,h ch laboratories company limited,cheyham corporation limited
1,10051937-TST-MR,10117193,0.956102,c h laboratories company limited,cheyham corporation limited
2,10051937-TST-MR,12212206,0.956102,ch laboratories company limited,cheyham corporation limited
3,10051937-TST-MR,12106617,0.952661,aprontech co limited,cheyham corporation limited
4,10051937-TST-MR,10036617,0.952661,dehanchar corporation limited,cheyham corporation limited
5,10051937-TST-MR,10040198,0.952036,rich hope company limited,cheyham corporation limited
6,10051937-TST-MR,10203106,0.950654,heworth company limited,cheyham corporation limited
7,10051937-TST-MR,10136888,0.947255,shiruida machinery import export co ltd,cheyham corporation limited
8,10051937-TST-MR,10211735,0.946762,hareton company limited,cheyham corporation limited
9,10051937-TST-MR,10145409,0.945905,pastry horizon china co limited,cheyham corporation limited


In [185]:
df_test_new['editdistance'] = extract_editdistance(df_test_new.predicted_name.values, df_test_new.queried_name.values)

In [192]:
df_test_new['editdistance'] = df_test_new['editdistance'].astype(int)
df_test_new['cosine_score'] = df_test_new['cosine_score'].astype(float)
df_test_new['predicted_record_id'] = df_test_new['predicted_record_id'].astype(int)

In [188]:
group = df_xgb.groupby('queried_record_id').size().values

In [191]:
df_test_new[['predicted_record_id', 'cosine_score', 'editdistance']]

,predicted_record_id,cosine_score,editdistance
0,10110955,0.968152,19
1,10117193,0.956102,18
2,12212206,0.956102,17
3,12106617,0.952661,16
4,10036617,0.952661,6
5,10040198,0.952036,14
6,10203106,0.950654,13
7,10136888,0.947255,29
8,10211735,0.946762,14
9,10145409,0.945905,19


In [195]:
df_test_new = df_test_new.rename(columns={'cosine_score': 'score'})

In [196]:
predictions = ranker.predict(df_test_new[['predicted_record_id', 'score', 'editdistance']])

In [199]:
predictions

array([-5.081288 , -5.0598927, -5.0598927, ...,  1.2769742,  1.1719849,
        2.8864481], dtype=float32)

In [205]:
df_test_new['predictions'] = predictions
df_predictions = df_test_new[['queried_record_id', 'predicted_record_id', 'predictions']]
df_predictions

,queried_record_id,predicted_record_id,predictions
0,10051937-TST-MR,10110955,-5.081288
1,10051937-TST-MR,10117193,-5.059893
2,10051937-TST-MR,12212206,-5.059893
3,10051937-TST-MR,12106617,-5.059893
4,10051937-TST-MR,10036617,-3.567889
5,10051937-TST-MR,10040198,-5.059893
6,10051937-TST-MR,10203106,-5.059893
7,10051937-TST-MR,10136888,-5.081288
8,10051937-TST-MR,10211735,-5.059893
9,10051937-TST-MR,10145409,-5.081288


In [206]:
rec_pred = []
for (r,p) in zip(df_predictions.predicted_record_id, df_predictions.predictions):
    rec_pred.append((r, p))
rec_pred

[(10110955, -5.081287860870361),
 (10117193, -5.059892654418945),
 (12212206, -5.059892654418945),
 (12106617, -5.059892654418945),
 (10036617, -3.5678892135620117),
 (10040198, -5.059892654418945),
 (10203106, -5.059892654418945),
 (10136888, -5.081287860870361),
 (10211735, -5.059892654418945),
 (10145409, -5.081287860870361),
 (10110955, -5.081287860870361),
 (10117193, -5.081287860870361),
 (12212206, -5.081287860870361),
 (12108462, -5.081287860870361),
 (12104370, 2.903371572494507),
 (12107086, 2.903371572494507),
 (10125563, -5.081287860870361),
 (13011853, -5.081287860870361),
 (10101237, -5.081287860870361),
 (10139756, -5.081287860870361),
 (10110955, -5.081287860870361),
 (10060448, -5.059892654418945),
 (10200687, -5.059892654418945),
 (10206976, -5.081287860870361),
 (12156474, -5.059892654418945),
 (10155126, -5.081287860870361),
 (11009308, -5.081287860870361),
 (10062850, -5.059892654418945),
 (12004034, -5.081287860870361),
 (12096140, -5.081287860870361),
 (10110955,

In [207]:
df_predictions['rec_pred'] = rec_pred
df_predictions

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,queried_record_id,predicted_record_id,predictions,rec_pred
0,10051937-TST-MR,10110955,-5.081288,"(10110955, -5.081287860870361)"
1,10051937-TST-MR,10117193,-5.059893,"(10117193, -5.059892654418945)"
2,10051937-TST-MR,12212206,-5.059893,"(12212206, -5.059892654418945)"
3,10051937-TST-MR,12106617,-5.059893,"(12106617, -5.059892654418945)"
4,10051937-TST-MR,10036617,-3.567889,"(10036617, -3.5678892135620117)"
5,10051937-TST-MR,10040198,-5.059893,"(10040198, -5.059892654418945)"
6,10051937-TST-MR,10203106,-5.059893,"(10203106, -5.059892654418945)"
7,10051937-TST-MR,10136888,-5.081288,"(10136888, -5.081287860870361)"
8,10051937-TST-MR,10211735,-5.059893,"(10211735, -5.059892654418945)"
9,10051937-TST-MR,10145409,-5.081288,"(10145409, -5.081287860870361)"


In [211]:
group_queried = df_predictions[['queried_record_id', 'rec_pred']].groupby('queried_record_id').apply(lambda x: list(x['rec_pred']))
group_queried

queried_record_id
10000003-TST-MR        [(10093582, -5.059892654418945), (10113802, -5...
10000008-TST-M         [(10029584, -5.059892654418945), (10128448, -5...
10000010-TST-CP        [(10155246, -5.059892654418945), (10108279, -3...
10000013-TST-MR        [(10127126, -5.059892654418945), (10202426, -5...
10000016-TST-MR        [(10134861, -5.081287860870361), (10034520, -5...
10000017-TST-MR        [(10152573, -3.5810060501098633), (12131717, -...
10000018-T4-TST-CP     [(10014054, -2.6996192932128906), (10187290, -...
10000018-TST-M         [(10014054, -2.6996192932128906), (10187290, -...
10000020-NV0-TST-CP    [(10099208, -5.059892654418945), (10070569, -3...
10000020-NV1-TST-M     [(10099208, -5.059892654418945), (10070569, -3...
10000022-TST-CP        [(10104990, -5.059892654418945), (10047729, -5...
10000023-NV2-TST-CP    [(10007187, -5.059892654418945), (10027264, -5...
10000024-TST-MR        [(10110183, -5.059892654418945), (12136485, -5...
10000025-TST-MR        [(10179644

In [225]:
df_predictions = pd.DataFrame(group_queried).reset_index().rename(columns={0 : 'rec_pred'})

In [229]:
df_predictions

,queried_record_id,rec_pred
0,10000003-TST-MR,"[(10093582, -5.059892654418945), (10113802, -5..."
1,10000008-TST-M,"[(10029584, -5.059892654418945), (10128448, -5..."
2,10000010-TST-CP,"[(10155246, -5.059892654418945), (10108279, -3..."
3,10000013-TST-MR,"[(10127126, -5.059892654418945), (10202426, -5..."
4,10000016-TST-MR,"[(10134861, -5.081287860870361), (10034520, -5..."
5,10000017-TST-MR,"[(10152573, -3.5810060501098633), (12131717, -..."
6,10000018-T4-TST-CP,"[(10014054, -2.6996192932128906), (10187290, -..."
7,10000018-TST-M,"[(10014054, -2.6996192932128906), (10187290, -..."
8,10000020-NV0-TST-CP,"[(10099208, -5.059892654418945), (10070569, -3..."
9,10000020-NV1-TST-M,"[(10099208, -5.059892654418945), (10070569, -3..."


In [240]:
def reorder_preds(preds):
    sorted_list = []
    for i in range(len(preds)):
        l = sorted(preds[i], key=lambda t: t[1], reverse=True)
        l = [x[0] for x in l]
        sorted_list.append(l)
    return sorted_list

In [239]:
df_predictions['ordered_preds'] = reorder_preds(df_predictions.rec_pred.values)
df_predictions

,queried_record_id,rec_pred,ordered_preds
0,10000003-TST-MR,"[(10093582, -5.059892654418945), (10113802, -5...","[(10004617, -2.6996192932128906), (10093582, -..."
1,10000008-TST-M,"[(10029584, -5.059892654418945), (10128448, -5...","[(10043172, 3.862312078475952), (10029584, -5...."
2,10000010-TST-CP,"[(10155246, -5.059892654418945), (10214153, -5...","[(10000010, 3.7945632934570312), (10057797, -0..."
3,10000013-TST-MR,"[(10027275, -5.081287860870361), (10127126, -5...","[(10153043, -1.9901454448699951), (12136105, -..."
4,10000016-TST-MR,"[(10134861, -5.081287860870361), (10034520, -5...","[(10147072, -0.327970027923584), (10012410, -2..."
5,10000017-TST-MR,"[(10079062, -5.059892654418945), (10106489, -5...","[(10170427, -2.7482330799102783), (10196127, -..."
6,10000018-T4-TST-CP,"[(10075075, -5.059892654418945), (12115160, -5...","[(10000018, 3.7945632934570312), (10000069, 1...."
7,10000018-TST-M,"[(10075075, -5.059892654418945), (12115160, -5...","[(10000018, 3.7945632934570312), (10000069, 1...."
8,10000020-NV0-TST-CP,"[(10099208, -5.059892654418945), (10128438, -5...","[(10000020, 3.7945632934570312), (10094948, -0..."
9,10000020-NV1-TST-M,"[(10099208, -5.059892654418945), (10128438, -5...","[(10000020, 3.7945632934570312), (10094948, -0..."


In [241]:
df_predictions['only_preds'] = reorder_preds(df_predictions.rec_pred.values)
df_predictions

,queried_record_id,rec_pred,ordered_preds,only_preds
0,10000003-TST-MR,"[(10093582, -5.059892654418945), (10113802, -5...","[(10004617, -2.6996192932128906), (10093582, -...","[10004617, 10093582, 10113802, 10010930, 10002..."
1,10000008-TST-M,"[(10029584, -5.059892654418945), (10128448, -5...","[(10043172, 3.862312078475952), (10029584, -5....","[10043172, 10029584, 10128448, 10079344, 10107..."
2,10000010-TST-CP,"[(10155246, -5.059892654418945), (10214153, -5...","[(10000010, 3.7945632934570312), (10057797, -0...","[10000010, 10057797, 10005319, 10108279, 10021..."
3,10000013-TST-MR,"[(10027275, -5.081287860870361), (10127126, -5...","[(10153043, -1.9901454448699951), (12136105, -...","[10153043, 12136105, 10071664, 10147826, 10107..."
4,10000016-TST-MR,"[(10134861, -5.081287860870361), (10034520, -5...","[(10147072, -0.327970027923584), (10012410, -2...","[10147072, 10012410, 10002588, 10006940, 10113..."
5,10000017-TST-MR,"[(10079062, -5.059892654418945), (10106489, -5...","[(10170427, -2.7482330799102783), (10196127, -...","[10170427, 10196127, 12131717, 10152573, 10079..."
6,10000018-T4-TST-CP,"[(10075075, -5.059892654418945), (12115160, -5...","[(10000018, 3.7945632934570312), (10000069, 1....","[10000018, 10000069, 12180349, 10014054, 10187..."
7,10000018-TST-M,"[(10075075, -5.059892654418945), (12115160, -5...","[(10000018, 3.7945632934570312), (10000069, 1....","[10000018, 10000069, 12180349, 10014054, 10187..."
8,10000020-NV0-TST-CP,"[(10099208, -5.059892654418945), (10128438, -5...","[(10000020, 3.7945632934570312), (10094948, -0...","[10000020, 10094948, 10140964, 10070569, 10113..."
9,10000020-NV1-TST-M,"[(10099208, -5.059892654418945), (10128438, -5...","[(10000020, 3.7945632934570312), (10094948, -0...","[10000020, 10094948, 10140964, 10070569, 10113..."


In [242]:
df_predictions = df_predictions[['queried_record_id', 'only_preds']].rename(columns={'only_preds': 'predicted_record_id'})
df_predictions

,queried_record_id,predicted_record_id
0,10000003-TST-MR,"[10004617, 10093582, 10113802, 10010930, 10002..."
1,10000008-TST-M,"[10043172, 10029584, 10128448, 10079344, 10107..."
2,10000010-TST-CP,"[10000010, 10057797, 10005319, 10108279, 10021..."
3,10000013-TST-MR,"[10153043, 12136105, 10071664, 10147826, 10107..."
4,10000016-TST-MR,"[10147072, 10012410, 10002588, 10006940, 10113..."
5,10000017-TST-MR,"[10170427, 10196127, 12131717, 10152573, 10079..."
6,10000018-T4-TST-CP,"[10000018, 10000069, 12180349, 10014054, 10187..."
7,10000018-TST-M,"[10000018, 10000069, 12180349, 10014054, 10187..."
8,10000020-NV0-TST-CP,"[10000020, 10094948, 10140964, 10070569, 10113..."
9,10000020-NV1-TST-M,"[10000020, 10094948, 10140964, 10070569, 10113..."


In [260]:
missing_values = {'queried_record_id' : ['12026587-TST-MR', '13009531-TST-MR', '12091134-TST-M', '12091134-NV0-TST-CP'], 
                 'predicted_record_id': [10111147, 10111147, 10111147, 10111147]}
missing_df = pd.DataFrame(missing_values)
missing_df

,queried_record_id,predicted_record_id
0,12026587-TST-MR,10111147
1,13009531-TST-MR,10111147
2,12091134-TST-M,10111147
3,12091134-NV0-TST-CP,10111147


In [256]:
new_col = []
for t in tqdm(df_predictions.predicted_record_id):
    new_col.append(' '.join([str(x) for x in t]))
new_col

['10004617 10093582 10113802 10010930 10002385 10022487 10097822 10090422 10077754 10205293',
 '10043172 10029584 10128448 10079344 10107250 10007755 10080769 10211873 10206650 10186437',
 '10000010 10057797 10005319 10108279 10021610 10198226 10124067 10155246 10214153 10186594',
 '10153043 12136105 10071664 10147826 10107777 10189807 10127126 10202426 10139664 10027275',
 '10147072 10012410 10002588 10006940 10113057 10002937 10000462 10134861 10034520 10186211',
 '10170427 10196127 12131717 10152573 10079062 10106489 10125867 10092259 10087602 10126538',
 '10000018 10000069 12180349 10014054 10187290 10160621 10075075 12115160 12188120 10100274',
 '10000018 10000069 12180349 10014054 10187290 10160621 10075075 12115160 12188120 10100274',
 '10000020 10094948 10140964 10070569 10113164 10062721 10173393 10099208 10128438 10127211',
 '10000020 10094948 10140964 10070569 10113164 10062721 10173393 10099208 10128438 10127211',
 '10000022 10213858 10104990 10047729 10056965 10193719 1100

In [264]:
df_predictions.predicted_record_id = new_col
df_predictions = pd.concat([df_predictions, missing_df])
df_predictions

,queried_record_id,predicted_record_id
0,10000003-TST-MR,10004617 10093582 10113802 10010930 10002385 1...
1,10000008-TST-M,10043172 10029584 10128448 10079344 10107250 1...
2,10000010-TST-CP,10000010 10057797 10005319 10108279 10021610 1...
3,10000013-TST-MR,10153043 12136105 10071664 10147826 10107777 1...
4,10000016-TST-MR,10147072 10012410 10002588 10006940 10113057 1...
5,10000017-TST-MR,10170427 10196127 12131717 10152573 10079062 1...
6,10000018-T4-TST-CP,10000018 10000069 12180349 10014054 10187290 1...
7,10000018-TST-M,10000018 10000069 12180349 10014054 10187290 1...
8,10000020-NV0-TST-CP,10000020 10094948 10140964 10070569 10113164 1...
9,10000020-NV1-TST-M,10000020 10094948 10140964 10070569 10113164 1...


In [263]:
#df_predictions = df_predictions.reset_index(drop=True)
#missing_idx = df_predictions[-5:].index
#df_predictions = df_predictions.drop(missing_idx)
#df_predictions

,queried_record_id,predicted_record_id
0,10000003-TST-MR,10004617 10093582 10113802 10010930 10002385 1...
1,10000008-TST-M,10043172 10029584 10128448 10079344 10107250 1...
2,10000010-TST-CP,10000010 10057797 10005319 10108279 10021610 1...
3,10000013-TST-MR,10153043 12136105 10071664 10147826 10107777 1...
4,10000016-TST-MR,10147072 10012410 10002588 10006940 10113057 1...
5,10000017-TST-MR,10170427 10196127 12131717 10152573 10079062 1...
6,10000018-T4-TST-CP,10000018 10000069 12180349 10014054 10187290 1...
7,10000018-TST-M,10000018 10000069 12180349 10014054 10187290 1...
8,10000020-NV0-TST-CP,10000020 10094948 10140964 10070569 10113164 1...
9,10000020-NV1-TST-M,10000020 10094948 10140964 10070569 10113164 1...


In [265]:
df_predictions.to_csv('xgb_sub2.csv', index=False)